In [14]:
import numpy as np
import pypsa

In [15]:
# marginal costs in EUR/MWh
marginal_cost = {"Wind": 0, "Hydro": 0, "Coal": 30, "Gas": 60, "Oil": 80}

# power plant capacities (nominal powers in MW) in each country (not necessarily realistic)
power_plant_p_nom_ = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000},
    "Mozambique": {
        "Hydro": 1200,
    },
    "Swaziland": {
        "Hydro": 600,
    },
}

# transmission capacities in MW (not necessarily realistic)
transmission = {
    "South Africa": {"Mozambique": 500, "Swaziland": 250},
    "Mozambique": {"Swaziland": 100},
}

# country electrical loads in MW (not necessarily realistic)
loads = {"South Africa": 42000, "Mozambique": 650, "Swaziland": 250}

In [16]:
country="South Africa"
network=pypsa.Network()
network.add("Bus",country)
for tech in power_plant_p_nom_[country]:
    network.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom_[country][tech],
        marginal_cost=marginal_cost[tech],
    )
network.add("Load", f"{country} load", bus=country, p_set=loads[country])

Index(['South Africa load'], dtype='object')

In [18]:
# Run optimisation to determine market dispatch
network.optimize()

Index(['South Africa'], dtype='object', name='Bus')
Index(['South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 4 primals, 9 duals
Objective: 1.29e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


('ok', 'optimal')

In [19]:
network.loads_t.p

Load,South Africa load
snapshot,
now,42000.0


In [21]:
network.generators_t.p

Generator,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,
now,35000.0,3000.0,4000.0,-0.0


In [22]:
network.buses_t.marginal_price

Bus,South Africa
snapshot,
now,60.0
